In [3]:
pip install transformers


     ---------------------------------------- 0.0/137.6 kB ? eta -:--:--
     -- ------------------------------------- 10.2/137.6 kB ? eta -:--:--
     ---------------- -------------------- 61.4/137.6 kB 812.7 kB/s eta 0:00:01
     -------------------------------------- 137.6/137.6 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.4/9.0 MB 7.6 MB/s eta 0:00:02
   --- ------------------------------------ 0.8/9.0 MB 8.5 MB/s eta 0:00:01
   ----- ---------------------------------- 1.3/9.0 MB 8.9 MB/s eta 0:00:01
   ------- -------------------------------- 1.8/9.0 MB 9.3 MB/s eta 0:00:01
   ---------- ----------------------------- 2.3/9.0 MB 9.9 MB/s eta 0:00:01
   ----------- ---------------------------- 2.5/9.0 MB 10.1 MB/s eta 0:00:01
   ------------ --------------------------- 2.9/9.0 MB 8.7 MB/s eta 0:00:01
   ------------- -------------------------- 3.1/9.0 MB 8.2 MB/s eta 0:00:01
   -----------

In [11]:
pip install transformers[torch] -U

   ---------------------------------------- 0.0/297.6 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/297.6 kB 435.7 kB/s eta 0:00:01
   ----------------------------------- ---- 266.2/297.6 kB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 297.6/297.6 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


In [138]:
pip install ray[tune]


   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   --------------- ----------------------- 41.0/101.7 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 101.7/101.7 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.4 MB ? eta -:--:--
    --------------------------------------- 0.4/25.4 MB 11.2 MB/s eta 0:00:03
   - -------------------------------------- 0.9/25.4 MB 10.9 MB/s eta 0:00:03
   - -------------------------------------- 1.2/25.4 MB 10.8 MB/s eta 0:00:03
   -- ------------------------------------- 1.8/25.4 MB 10.6 MB/s eta 0:00:03
   --- ------------------------------------ 2.3/25.4 MB 10.5 MB/s eta 0:00:03
   ---- ----------------------------------- 2.9/25.4 MB 10.7 MB/s eta 0:00:03
   ----- ---------------------------------- 3.4/25.4 MB 11.5 MB/s eta 0:00:02
   ------ --------------------------------- 3.8/25.4 MB 10.6 MB/s eta 0:00:03
   ------ --------------------------------- 4.4/25.4 MB 10.8 MB/s eta 0:00

In [140]:
pip install -U ipywidgets

   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/139.4 kB ? eta -:--:--
   ----------- --------------------------- 41.0/139.4 kB 495.5 kB/s eta 0:00:01
   -------------------------------------- - 133.1/139.4 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- 139.4/139.4 kB 828.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   ---------------------------------------- 215.0/215.0 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/2.3 MB 7.6 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/2.3 MB 6.5 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.3 MB 4.6 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.3 MB 5.1 MB/s eta 0:00:01
   ----------------------- ---------------- 1.3/2.3 MB 6.1 MB/s eta 0:00:01
   --------------

In [171]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from ray import tune

In [172]:
def train_model(config):
    # Load the dataset
    data_path = r'C:\Users\HP\Desktop\Eitacies\Detection of Python Code presence in English sentences\Shuffled Training Data.csv'
    data = pd.read_csv(data_path)
    
    # Split the dataset into training and testing sets
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    train_encodings = tokenizer(train_data['Text'].tolist(), truncation=True, padding=True, max_length=128)
    test_encodings = tokenizer(test_data['Text'].tolist(), truncation=True, padding=True, max_length=128)
    
    class TextDataset(Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)
    
    train_dataset = TextDataset(train_encodings, train_data['Class'].tolist())
    test_dataset = TextDataset(test_encodings, test_data['Class'].tolist())
    
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy='steps',
        learning_rate=config["learning_rate"],
        per_device_train_batch_size=int(config["train_batch_size"]),
        num_train_epochs=config["num_train_epochs"],
        weight_decay=config["weight_decay"],
        logging_dir='./logs',
        logging_steps=50,
    )
    
    # Initialize the model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    
    # Initialize the Trainer
    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
    )
    
    # Train and evaluate the model
    trainer.train()
    results = trainer.evaluate()
    

In [173]:
# Additionally, you may want to compute predictions for further manual metrics computation
predictions_output = trainer.predict(test_dataset)
predictions = np.argmax(predictions_output.predictions, axis=1)
labels = predictions_output.label_ids

In [174]:
# Calculate accuracy
accuracy = accuracy_score(labels, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.4375


In [175]:
# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.4375
Recall: 1.0
F1 Score: 0.6086956521739131


In [158]:
# Generate confusion matrix
conf_matrix = confusion_matrix(labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[0 9]
 [0 7]]


In [159]:
# Load the unseen data
data_path = r'C:\Users\HP\Desktop\Eitacies\Detection of Python Code presence in English sentences\Unseen Data.csv'
data = pd.read_csv(data_path)
texts = data['Text'].tolist()

In [160]:
#Instantiate the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [161]:
encodings = tokenize_data(texts)

In [162]:
unseen_dataset = TextDataset(encodings)

TypeError: __init__() missing 1 required positional argument: 'labels'

In [163]:
# Load the trained model
model_directory = "saved_model"  # Adjust the path if needed
model = BertForSequenceClassification.from_pretrained(model_directory, num_labels=2)

In [164]:
# Put the model in evaluation mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [166]:
from torch.utils.data import DataLoader

In [167]:
# Create a DataLoader for the unseen dataset
dataloader = DataLoader(unseen_dataset, batch_size=16)  # Adjust batch size as needed

In [168]:
# Inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [169]:
predictions = []
with torch.no_grad():
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        predictions.extend(torch.argmax(outputs.logits, dim=1).tolist())

In [170]:
# Print the predictions alongside the text
for text, pred in zip(texts, predictions):
    print(f"Text: {text}\nPredicted Class: {pred}\n")

Text: print("Hello, world!")
Predicted Class: 1

Text: def add(x, y): return x + y
Predicted Class: 1

Text: for i in range(10): print(i)
Predicted Class: 1

Text: import numpy as np; np.zeros((2,2))
Predicted Class: 1

Text: if __name__ == "__main__": print("This is a script.")
Predicted Class: 1

Text: Walking my dog in the park is my favorite part of the day.
Predicted Class: 0

Text: I love to read historical novels on rainy days.
Predicted Class: 0

Text: Chocolate cake is best served with a scoop of vanilla ice cream.
Predicted Class: 1

Text: Yesterday, I watched a documentary about ancient civilizations.
Predicted Class: 0

Text: Gardening is a relaxing hobby that connects you with nature.
Predicted Class: 0

Text: x = [i for i in range(10) if i % 2 == 0]
Predicted Class: 1

Text: def greet(name): return f"Hello, {name}!"
Predicted Class: 1

Text: import random; random.seed(42)
Predicted Class: 1

Text: x = {'key': 'value'}; print(x['key'])
Predicted Class: 1

Text: try: x = 1 

(raylet) [2024-04-24 20:43:08,818 C 36520 22472] (raylet.exe) node_manager.cc:537: GCS consider this node to be dead. This may happen when GCS is not backed by a DB and restarted or there is data loss in the DB.
(raylet) *** StackTrace Information ***
(raylet) unknown
(raylet) BaseThreadInitThunk
(raylet) RtlUserThreadStart
(raylet) 
